<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/new_features_impacto_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
### Tutorial: Generación de Nuevas Características para Mejorar la Precisión de un Modelo

# Caso de estudio: Predicción de supervivencia en el Titanic
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Cargar el conjunto de datos Titanic
datos = sns.load_dataset('titanic')

# Renombrar columnas a español
datos.rename(columns={
    'pclass': 'clase',
    'sex': 'sexo',
    'age': 'edad',
    'sibsp': 'hermanos_esposos',
    'parch': 'padres_hijos',
    'fare': 'tarifa',
    'embarked': 'embarque',
    'survived': 'sobrevivio'
}, inplace=True)


In [26]:
# Manejo de valores faltantes
from sklearn.impute import SimpleImputer
imputador = SimpleImputer(strategy='most_frequent')
datos['edad'] = imputador.fit_transform(datos[['edad']])
datos['embarque'] = imputador.fit_transform(datos[['embarque']])[:, 0] # Reshape to 1D array
datos.dropna(subset=['embarque'], inplace=True)

# Codificación de variables categóricas
from sklearn.preprocessing import LabelEncoder
codificador = LabelEncoder()
datos['sexo'] = codificador.fit_transform(datos['sexo'])
datos['embarque'] = codificador.fit_transform(datos['embarque'])

# Definir las características y el objetivo
caracteristicas = ['clase', 'sexo', 'edad', 'hermanos_esposos', 'padres_hijos', 'tarifa', 'embarque']
objetivo = 'sobrevivio'

# Dividir los datos en conjuntos de entrenamiento y prueba
X = datos[caracteristicas]
y = datos[objetivo]
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo base sin nuevas características
modelo_base = RandomForestClassifier(random_state=42)
modelo_base.fit(X_entreno, y_entreno)
y_pred_base = modelo_base.predict(X_prueba)

precision_base = accuracy_score(y_prueba, y_pred_base)
print(f"Precisión del modelo base: {precision_base:.4f}")

Precisión del modelo base: 0.7836


In [27]:
# Generación de nuevas características
datos['tamano_familia'] = datos['hermanos_esposos'] + datos['padres_hijos'] + 1
datos['esta_solo'] = (datos['tamano_familia'] == 1).astype(int)

# Actualizar las características para incluir las nuevas
caracteristicas_actualizadas = ['clase', 'sexo', 'edad', 'tarifa', 'embarque', 'tamano_familia', 'esta_solo']
X = datos[caracteristicas_actualizadas]
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo con nuevas características
modelo_mejorado = RandomForestClassifier(random_state=42)
modelo_mejorado.fit(X_entreno, y_entreno)
y_pred_mejorado = modelo_mejorado.predict(X_prueba)

precision_mejorada = accuracy_score(y_prueba, y_pred_mejorado)
print(f"Precisión con nuevas características: {precision_mejorada:.4f}")

# Comparación de resultados
mejora = precision_mejorada - precision_base
print(f"Mejora en la precisión: {mejora:.4f}")

Precisión con nuevas características: 0.7948
Mejora en la precisión: 0.0112
